In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from load import load_data
from load import series_to_supervised
from load import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from load import get_measures
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")
rcParams['figure.figsize'] = 15, 5 # tamanho das figuras



def multi_step_forecast(data, n):
    n_test = outs = n
    train, test = train_test_split(data, n_test)
    train.dropna(inplace = True)
    response_vars = data.columns[-(outs):]
    predictions = list()
    for h, response in enumerate(response_vars):
        cols = [x for x in data.columns[:data.shape[1] - outs]]
        cols.append(response)
        data_ = train[cols]
        nrows = data_.shape[0]
        data_ = data_.iloc[:nrows-h, :] 
        data_X, data_y = data_.iloc[:, :-1], data_.iloc[:, -1]
        model = lgb.LGBMRegressor(objective='regression', n_estimators=1000)
        model.fit(data_X, data_y)
        testX, testy = test.reset_index(drop=True).loc[0, :"var1(t-1)"], test.reset_index(drop=True).loc[0, response]
        pred = model.predict([testX])[0]
        print(f"Predicting {response}\n  > expected: {testy}, predicted: {pred}")
        predictions.append(pred)
    measures = get_measures(pd.Series(predictions), test["var1(t)"])
    df_measures = pd.DataFrame([measures])
    return predictions, df_measures, test, data_X, data_y, testX

df_load = load_data()
df_weather = pd.read_csv("../data/weather_daily_data.csv", parse_dates=["DATA"])
df_weather = df_weather[df_weather.DATA.isin(df_load.index)]
df_load = df_load[df_weather.DATA.min():df_weather.DATA.max()] 



df_load_2 = df_load.reset_index()
df_merged = pd.merge(df_weather, df_load_2, left_on = "DATA", right_on = "date", how = "outer")
df_merged.drop("date", axis = 1, inplace = True)

df_merged.dropna(how = "all", inplace = True)
df_merged.sort_values(by = "DATA", inplace = True)
df_merged.load_mwmed = df_merged.load_mwmed.interpolate(method="linear")


lag = 15    # lags das variáveis climáticas e de carga que serão utilizadas
n_test = outs = 1 # horizonte de previsão e teste (= número de modelos)

df_load_3 = df_merged.load_mwmed
df_load_3.index = df_merged.DATA
df_load_3 = df_load_3["2008-01-01":] # TESTE: MAPE PASSOU DE 3,2 PARA 3,0
values = df_load_3.values.tolist()
data1 = series_to_supervised(values, n_in = lag, n_out=outs, dropnan=False)

data2 = pd.DataFrame()
df_weather.set_index("DATA", inplace=True) # TESTE: MAPE PASSOU DE 3,2 PARA 3,0
df_weather = df_weather["2008-01-01":] # TESTE: MAPE PASSOU DE 3,2 PARA 3,0
for col in df_weather.columns:
    if col == "DATA":
        continue
    else:
        values = df_weather[col].values.tolist()
        df_ = series_to_supervised(values, n_in = lag, dropnan=False)
        df_.drop("var1(t)", axis = 1, inplace = True) # the response variable is the load dataframe
        df_.columns = [f"{x}_{col}" for x in df_.columns]
        data2 = pd.concat([data2, df_], axis = 1)
        
df_weather_load = pd.concat([data2, data1], axis = 1)


pred, measures, test, X, y, testX = multi_step_forecast(df_weather_load, outs)

Predicting var1(t)
  > expected: 12162.75679167, predicted: 11780.495079320684


In [18]:
from sklearn.feature_selection import RFE

In [19]:
X

,var1(t-15)_PRECIPITACAO_PR,var1(t-14)_PRECIPITACAO_PR,var1(t-13)_PRECIPITACAO_PR,var1(t-12)_PRECIPITACAO_PR,var1(t-11)_PRECIPITACAO_PR,var1(t-10)_PRECIPITACAO_PR,var1(t-9)_PRECIPITACAO_PR,var1(t-8)_PRECIPITACAO_PR,var1(t-7)_PRECIPITACAO_PR,var1(t-6)_PRECIPITACAO_PR,...,var1(t-10),var1(t-9),var1(t-8),var1(t-7),var1(t-6),var1(t-5),var1(t-4),var1(t-3),var1(t-2),var1(t-1)
15,65.4,136.0,171.8,85.4,120.8,34.0,16.2,0.4,25.4,11.6,...,6930.564167,9086.430417,9437.825000,9535.760417,9946.955417,9684.091667,8194.232083,6997.824167,9407.491250,9897.250000
16,136.0,171.8,85.4,120.8,34.0,16.2,0.4,25.4,11.6,119.8,...,9086.430417,9437.825000,9535.760417,9946.955417,9684.091667,8194.232083,6997.824167,9407.491250,9897.250000,9960.365417
17,171.8,85.4,120.8,34.0,16.2,0.4,25.4,11.6,119.8,128.2,...,9437.825000,9535.760417,9946.955417,9684.091667,8194.232083,6997.824167,9407.491250,9897.250000,9960.365417,9751.891667
18,85.4,120.8,34.0,16.2,0.4,25.4,11.6,119.8,128.2,59.4,...,9535.760417,9946.955417,9684.091667,8194.232083,6997.824167,9407.491250,9897.250000,9960.365417,9751.891667,9676.098333
19,120.8,34.0,16.2,0.4,25.4,11.6,119.8,128.2,59.4,52.4,...,9946.955417,9684.091667,8194.232083,6997.824167,9407.491250,9897.250000,9960.365417,9751.891667,9676.098333,8231.710417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5259,28.8,5.2,2.2,73.2,15.0,1.6,0.2,0.2,1.4,0.4,...,11424.087542,11964.909375,12269.051375,12021.415458,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,12186.721375
5260,5.2,2.2,73.2,15.0,1.6,0.2,0.2,1.4,0.4,0.2,...,11964.909375,12269.051375,12021.415458,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,12186.721375,12482.523708
5261,2.2,73.2,15.0,1.6,0.2,0.2,1.4,0.4,0.2,0.0,...,12269.051375,12021.415458,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,12186.721375,12482.523708,12520.803833
5262,73.2,15.0,1.6,0.2,0.2,1.4,0.4,0.2,0.0,0.0,...,12021.415458,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,12186.721375,12482.523708,12520.803833,10525.490875


In [20]:
y

15       9960.365417
16       9751.891667
17       9676.098333
18       8231.710417
19       6804.990000
            ...     
5259    12482.523708
5260    12520.803833
5261    10525.490875
5262     9074.211250
5263    11648.709583
Name: var1(t), Length: 5249, dtype: float64

In [21]:
estimator = lgb.LGBMRegressor(objective='regression', n_estimators=1000)
selector = RFE(estimator, n_features_to_select=20, step=1)
selector = selector.fit(X, y)
pred = selector.predict([testX])[0]
pred

In [22]:
pred = selector.predict([testX])[0]
pred

11999.657148589982

In [30]:
len(X.columns[selector.support_]) # variáveis selecionadas

20

In [35]:
selected_vars = [x for x in X.columns[selector.support_]] # variáveis selecionadas
selected_vars

['var1(t-8)_PRESSAO_ATM_PR',
 'var1(t-15)_VELOCIDADE_VENTO_PR',
 'var1(t-9)_VELOCIDADE_VENTO_PR',
 'var1(t-3)_VELOCIDADE_VENTO_PR',
 'var1(t-5)_PRESSAO_ATM_RS',
 'var1(t-1)_PRESSAO_ATM_RS',
 'var1(t-11)_TEMP_AR_BULBO_SECO_RS',
 'var1(t-8)_UMIDADE_RELATIVA_AR_RS',
 'var1(t-14)_VELOCIDADE_VENTO_RS',
 'var1(t-6)_VELOCIDADE_VENTO_RS',
 'var1(t-15)_PRECIPITACAO_SC',
 'var1(t-9)_PRECIPITACAO_SC',
 'var1(t-4)_TEMP_ORVALHO_SC',
 'var1(t-1)_UMIDADE_RELATIVA_AR_SC',
 'var1(t-14)',
 'var1(t-12)',
 'var1(t-9)',
 'var1(t-7)',
 'var1(t-3)',
 'var1(t-1)']

In [ ]:
selected_vars.sort()
selected_vars

In [8]:
import json

vars = ['var1(t-1)',
 'var1(t-1)_PRESSAO_ATM_RS',
 'var1(t-1)_UMIDADE_RELATIVA_AR_SC',
 'var1(t-11)_TEMP_AR_BULBO_SECO_RS',
 'var1(t-12)',
 'var1(t-14)',
 'var1(t-14)_VELOCIDADE_VENTO_RS',
 'var1(t-15)_PRECIPITACAO_SC',
 'var1(t-15)_VELOCIDADE_VENTO_PR',
 'var1(t-3)',
 'var1(t-3)_VELOCIDADE_VENTO_PR',
 'var1(t-4)_TEMP_ORVALHO_SC',
 'var1(t-5)_PRESSAO_ATM_RS',
 'var1(t-6)_VELOCIDADE_VENTO_RS',
 'var1(t-7)',
 'var1(t-8)_PRESSAO_ATM_PR',
 'var1(t-8)_UMIDADE_RELATIVA_AR_RS',
 'var1(t-9)',
 'var1(t-9)_PRECIPITACAO_SC',
 'var1(t-9)_VELOCIDADE_VENTO_PR']

d = dict()
for x,y in enumerate(range(10)):
    d[f"h_{x}"] = vars

# escreve json
with open('selected_vars.json', 'w') as f:
    json.dump(d, f)

# lê json
with open('selected_vars.json') as f:
    my_dict = json.load(f)
my_dict

{'h_0': ['var1(t-1)',
  'var1(t-1)_PRESSAO_ATM_RS',
  'var1(t-1)_UMIDADE_RELATIVA_AR_SC',
  'var1(t-11)_TEMP_AR_BULBO_SECO_RS',
  'var1(t-12)',
  'var1(t-14)',
  'var1(t-14)_VELOCIDADE_VENTO_RS',
  'var1(t-15)_PRECIPITACAO_SC',
  'var1(t-15)_VELOCIDADE_VENTO_PR',
  'var1(t-3)',
  'var1(t-3)_VELOCIDADE_VENTO_PR',
  'var1(t-4)_TEMP_ORVALHO_SC',
  'var1(t-5)_PRESSAO_ATM_RS',
  'var1(t-6)_VELOCIDADE_VENTO_RS',
  'var1(t-7)',
  'var1(t-8)_PRESSAO_ATM_PR',
  'var1(t-8)_UMIDADE_RELATIVA_AR_RS',
  'var1(t-9)',
  'var1(t-9)_PRECIPITACAO_SC',
  'var1(t-9)_VELOCIDADE_VENTO_PR'],
 'h_1': ['var1(t-1)',
  'var1(t-1)_PRESSAO_ATM_RS',
  'var1(t-1)_UMIDADE_RELATIVA_AR_SC',
  'var1(t-11)_TEMP_AR_BULBO_SECO_RS',
  'var1(t-12)',
  'var1(t-14)',
  'var1(t-14)_VELOCIDADE_VENTO_RS',
  'var1(t-15)_PRECIPITACAO_SC',
  'var1(t-15)_VELOCIDADE_VENTO_PR',
  'var1(t-3)',
  'var1(t-3)_VELOCIDADE_VENTO_PR',
  'var1(t-4)_TEMP_ORVALHO_SC',
  'var1(t-5)_PRESSAO_ATM_RS',
  'var1(t-6)_VELOCIDADE_VENTO_RS',
  'var1(t-7)',
